### TensorFlow 實作

使用 TensorFlow 來建立一個簡單的 MLP 模型。

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

# 下載並預處理 MNIST 資料集
# MNIST 是一個手寫數字的資料集，包含 60,000 張訓練圖像和 10,000 張測試圖像
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# 將圖像數據從 0-255 的範圍縮放到 0-1 之間，這樣可以加速模型的訓練
x_train, x_test = x_train / 255.0, x_test / 255.0

# 創建模型
# 使用 Sequential 模型，這是一個線性堆疊的模型
model = models.Sequential([
    # 將 28x28 的圖像展開成一維向量，這樣可以輸入到全連接層中
    layers.Flatten(input_shape=(28, 28)),
    # 全連接層，包含 128 個神經元，使用 ReLU 激活函數
    layers.Dense(128, activation='relu'),
    # Dropout 層，隨機丟棄 20% 的神經元，防止過擬合
    layers.Dropout(0.2),
    # 輸出層，包含 10 個神經元，對應 10 個類別，使用 softmax 激活函數
    layers.Dense(10, activation='softmax')
])

# 編譯模型
# 使用 Adam 優化器，損失函數為 sparse_categorical_crossentropy，評估指標為準確率
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 訓練模型
# 使用訓練數據進行訓練，訓練 5 個 epoch
model.fit(x_train, y_train, epochs=5)

# 評估模型
# 使用測試數據進行評估，並輸出測試準確率
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)


### PyTorch 實作
接下來，我們使用 PyTorch 實作同樣的 MLP 模型。

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 設置資料轉換
# 將圖像轉換為張量並標準化到 [-1, 1] 範圍
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# 下載並加載 MNIST 資料集
# MNIST 是一個手寫數字的資料集，包含 60,000 張訓練圖像和 10,000 張測試圖像
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 使用 DataLoader 將資料集分批次加載
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# 創建神經網路模型
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # 將 28x28 的圖像展開成一維向量
        self.flatten = nn.Flatten()
        # 全連接層，將 784 維輸入轉換為 128 維
        self.fc1 = nn.Linear(28 * 28, 128)
        # Dropout 層，隨機丟棄 20% 的神經元，防止過擬合
        self.dropout = nn.Dropout(0.2)
        # 全連接層，將 128 維輸入轉換為 10 維，對應 10 個類別
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        # 前向傳播
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        # 使用 softmax 激活函數，將輸出轉換為概率分佈
        return torch.softmax(x, dim=1)

# 初始化模型、損失函數與優化器
model = MLP()
# 使用交叉熵損失函數
criterion = nn.CrossEntropyLoss()
# 使用 Adam 優化器，學習率為 0.001
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練模型
def train(model, train_loader, criterion, optimizer, epochs=5):
    model.train()  # 設置模型為訓練模式
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()  # 清空梯度
            outputs = model(images)  # 前向傳播
            loss = criterion(outputs, labels)  # 計算損失
            loss.backward()  # 反向傳播
            optimizer.step()  # 更新參數
            running_loss += loss.item()  # 累加損失
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")  # 輸出每個 epoch 的平均損失

# 測試模型
def test(model, test_loader):
    model.eval()  # 設置模型為評估模式
    correct = 0
    total = 0
    with torch.no_grad():  # 禁用梯度計算
        for images, labels in test_loader:
            outputs = model(images)  # 前向傳播
            _, predicted = torch.max(outputs.data, 1)  # 獲取預測結果
            total += labels.size(0)  # 累加總樣本數
            correct += (predicted == labels).sum().item()  # 累加正確預測數
    print(f'Test Accuracy: {100 * correct / total}%')  # 輸出測試準確率

# 執行訓練與測試
train(model, train_loader, criterion, optimizer, epochs=5)
test(model, test_loader)


### **步驟說明：**

1. **資料處理：**
   - 我們從 TensorFlow 和 PyTorch 內建的 `MNIST` 資料集下載手寫數字圖像資料，並對它們進行標準化（將像素值縮放至 0-1 之間）。
   
2. **神經網路結構：**
   - 在 TensorFlow 和 PyTorch 中，我們建立了一個簡單的多層感知器（MLP）。首先將 28x28 的圖像展開成一個向量（784 維），接著通過一個隱藏層進行全連接，並應用 `ReLU` 激活函數。最後的輸出層有 10 個神經元，對應於 0-9 的手寫數字分類。

3. **訓練與評估：**
   - 模型會訓練 5 個 epochs，並對測試集進行評估。你將得到模型的準確度。

這是基本的 MLP 神經網路，未來的訓練我們將會進一步探索更複雜的架構，如卷積神經網路（CNN）和 YOLO 物件檢測模型。